In [1]:
import pandas as pd
import numpy as np
name = "billing_and_accruals_for_script_2024-05-17_10-51-20"

In [2]:
dtype_dict = {
    'КПП': str,
}

In [3]:
dataset = pd.read_excel('C:/Users/Boris/Downloads/' + name + '.xlsx', dtype=dtype_dict)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Название компании           9971 non-null   object        
 1   Дата                        3921 non-null   datetime64[ns]
 2   Личный кабинет              10000 non-null  int64         
 3   ЛС                          3879 non-null   object        
 4   ИНН                         4084 non-null   object        
 5   КПП                         2727 non-null   object        
 6   Номер договора              3879 non-null   object        
 7   Идентификатор номенклатуры  3879 non-null   float64       
 8   Название номенклатуры       3879 non-null   object        
 9   Стоимость                   3921 non-null   float64       
 10  Сумма                       3921 non-null   float64       
 11  Начисления partner          3307 non-null   float64    

In [5]:
dataset['КПП'] = dataset['КПП'].astype(str)

In [6]:
dataset.head(2)

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner
0,ТСЖ Буинская 38а,NaT,120062,NaN,7710140679,nan,NaN,NaN,NaN,NaN,NaN,NaN
1,Антон Гладышев,NaT,120063,NaN,246312458454,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dataset['Начисления partner'] = dataset['Начисления partner'].fillna(0)

In [8]:
dataset['Сумма-test'] = dataset['Сумма']

In [9]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test
0,ТСЖ Буинская 38а,NaT,120062,NaN,7710140679,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,Антон Гладышев,NaT,120063,NaN,246312458454,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,ООО Инкомтех,NaT,120065,NaN,1215185339,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,"ООО ""Видеоглаз Вэст""",NaT,120066,NaN,507406410220,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,ООО «Эко-Мастер»,NaT,120067,NaN,3444065421,344401001,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,127710,NaT,127710,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
9996,127711,NaT,127711,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
9997,127712,NaT,127712,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN
9998,127713,NaT,127713,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [10]:
for i in range(0,dataset.shape[0]):
    if i == 0:
        date = dataset.loc[0,'Дата']
        lk = dataset.loc[0,'Личный кабинет'] 
        dataset.loc[0,'ballance'] = 0
        dataset.loc[0,'ballance'] = dataset.loc[0,'Сумма'] - dataset.loc[0, 'Начисления partner']
        if dataset.loc[0,'ballance'] < 0:
            dataset.loc[0,'Сумма']  = 0
        if dataset.loc[0,'ballance'] >= 0:
                dataset.loc[0,'Сумма'] = dataset.loc[0,'ballance'] 
                dataset.loc[0,'ballance'] = 0
            
    else:
    
        if date == dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if date != dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] - dataset.loc[i, 'Начисления partner'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if lk != dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            lk = dataset.loc[i,'Личный кабинет']
            
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] \
                                        - dataset.loc[i, 'Начисления partner']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0



In [11]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test,ballance
0,ТСЖ Буинская 38а,NaT,120062,NaN,7710140679,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,Антон Гладышев,NaT,120063,NaN,246312458454,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,ООО Инкомтех,NaT,120065,NaN,1215185339,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,"ООО ""Видеоглаз Вэст""",NaT,120066,NaN,507406410220,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,ООО «Эко-Мастер»,NaT,120067,NaN,3444065421,344401001,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,127710,NaT,127710,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
9996,127711,NaT,127711,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
9997,127712,NaT,127712,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
9998,127713,NaT,127713,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [12]:
dataset.to_excel('C:/Users/Boris/Downloads/'+ name + '_test.xlsx', index=False)

In [13]:
dataset_ready = dataset.drop(['Начисления partner','Сумма-test','ballance'], axis= 1)

In [14]:
dataset_ready['Количество'] = round(dataset_ready['Сумма']/dataset_ready['Стоимость'])
dataset_ready['Количество'] = dataset_ready['Количество'].fillna(0) 

In [15]:
dataset_ready

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Количество
0,ТСЖ Буинская 38а,NaT,120062,NaN,7710140679,nan,NaN,NaN,NaN,NaN,NaN,0.0
1,Антон Гладышев,NaT,120063,NaN,246312458454,nan,NaN,NaN,NaN,NaN,NaN,0.0
2,ООО Инкомтех,NaT,120065,NaN,1215185339,nan,NaN,NaN,NaN,NaN,NaN,0.0
3,"ООО ""Видеоглаз Вэст""",NaT,120066,NaN,507406410220,nan,NaN,NaN,NaN,NaN,NaN,0.0
4,ООО «Эко-Мастер»,NaT,120067,NaN,3444065421,344401001,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,127710,NaT,127710,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0
9996,127711,NaT,127711,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0
9997,127712,NaT,127712,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0
9998,127713,NaT,127713,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,0.0


In [16]:
dataset_ready.to_excel('C:/Users/Boris/Downloads/'+ name + '_ready.xlsx', index=False)